<a href="https://colab.research.google.com/github/xhang24/xiaotong/blob/master/src/hw7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import scipy.stats as ss

#Consider : dSt=0.03Stdt + σSt^γdWt, S0=100.

class VanillaOption:
    def __init__(
        self,
        otype = 1, # 1: 'call'
                  # -1: 'put'
        strike = 110.,
        maturity = 1.,
        market_price = 10.,
        n=1000,
        N=1000,
        r=0.0475,
        y=1,
        vol=0.2):
      self.otype = otype
      self.strike = strike
      self.maturity = maturity
      self.market_price = market_price #this will be used for calibration
      self.step = N
      self.path = n
      self.interest=r
      self.y=y
      self.vol=vol            

    def Euler_CEV(self,S0):
      o_type=self.otype
      T=self.maturity
      N=self.step
      n=self.path
      K=self.strike
      r=self.interest
      y0=self.y
      vol0=self.vol
      sum_value=0
      w=T/N
      for j in range(n):
        S=S0
        for i in range(N):
          z=np.random.normal()
          S=S+0.03*S*w+vol0*(S**y0)*np.sqrt(w)*z
        if (S-K)*o_type >0 :
          sum_value+=(S-K)*o_type
      return np.exp(-r*T)*sum_value/n

In [0]:
option=VanillaOption(strike=97)
option.Euler_CEV(100)

11.470638211140733

10.807686187734152

11.467355466533505

In [0]:
class Gbm:
    def __init__(self,
                 init_state = 100.,
                 drift_ratio = .0475,
                 vol_ratio = .2
                ):
        self.init_state = init_state
        self.drift_ratio = drift_ratio
        self.vol_ratio = vol_ratio
        
    def bsm_price(self, vanilla_option):
        s0 = self.init_state
        sigma = self.vol_ratio
        r = self.drift_ratio
    
        otype = vanilla_option.otype
        k = vanilla_option.strike
        maturity = vanilla_option.maturity
    
        d1 = (np.log(s0 / k) + (r + 0.5 * sigma ** 2) 
              * maturity) / (sigma * np.sqrt(maturity))
        d2 = d1 - sigma * np.sqrt(maturity)
    
        return (otype * s0 * ss.norm.cdf(otype * d1) #line break needs parenthesis
                - otype * np.exp(-r * maturity) * k * ss.norm.cdf(otype * d2))


In [0]:
gbm1=Gbm(drift_ratio=0.03)
gbm1.bsm_price(option)

11.014613780922488

11.014613780922488

11.014613780922488

In [0]:
!git clone https://github.com/songqsh/20s_ma573.git
%cd 20s_ma573/src/

np_option_data = np.loadtxt('20optiondata2.dat', comments='#', delimiter=',')
print('>>>otype,        maturity,  strike,       option_price')
print(np_option_data)

Cloning into '20s_ma573'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 297 (delta 46), reused 44 (delta 20), pack-reused 213
Receiving objects: 100% (297/297), 2.15 MiB | 16.04 MiB/s, done.
Resolving deltas: 100% (155/155), done.
/content/20s_ma573/src
>>>otype,        maturity,  strike,       option_price
[[  1.           0.16666667  97.           5.32705461]
 [  1.           0.16666667  99.           3.86224255]
 [  1.           0.16666667 101.           2.7204371 ]
 [  1.           0.16666667 103.           2.1202793 ]
 [  1.           0.41666667  97.           7.23756307]
 [  1.           0.41666667  99.           5.95053461]
 [  1.           0.41666667 101.           5.2640122 ]
 [  1.           0.41666667 103.           4.97493422]]
Cloning into '20s_ma573'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100%

In [0]:
def maturity_choice(mat_choice):
  option_data=np_option_data[np_option_data[:,1] == mat_choice]
  return option_data

def op_list(option):
  num_row = option.shape[0]
  option_list = []
  for i in range(num_row):
    option1 = VanillaOption(
        
        otype = option[i,0],
        strike = option[i,2],
        maturity = option[i,1],
        market_price = option[i,3]
    )   
    option_list.append(option1)
  return option_list

In [0]:
op_list1 = op_list(maturity_choice(5/12))

In [0]:
import scipy.optimize as so

def error_function(vol_y):
  error = 0
  for i in np.arange(len(op_list1)):
    op_list1[i].y=vol_y[1]
    op_list1[i].vol=vol_y[0]
    error = error + ((op_list1[i]).Euler_CEV(100) - (op_list1[i]).market_price)**2
  return error

initial_guess=np.ones(2)
initial_guess[0]=0.1
so.fmin(error_function,initial_guess,xtol=0.0001,ftol=0.0001,maxiter=200,maxfun=200)